#One Time Run

In [13]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
#loading data
data_raw = pd.read_excel('DATA-SVR.xlsx')
data_raw = data_raw.dropna(axis=1, how='all')

In [65]:
def normalisasi(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    return scaler.transform(data), scaler

def matrixHessian(data_1, data_2, lamda, d):
    xT = data_1.T
    # Rumus Sesuai Fungsi Kernel Polynomial (Pers 3.11)
    hessian_x_poly = (np.dot(data_2, xT) + 1) **d + lamda**2  
    return hessian_x_poly 

# Sequential Learning
# Goal : Nilai a* dan a
def sequentialLearning(data_train, datay_train, matrix_hessian, epsilon, C, cLR, maxIter = 100):
    z = 1
    
    a_star_init = list(np.zeros(data_train.shape[0]))
    a_init = list(np.zeros(data_train.shape[0]))

    hasilDalfaS = list(np.zeros(data_train.shape[0]))
    hasilDalfa = list(np.zeros(data_train.shape[0]))

    UpdateAlfaS = list(np.zeros(data_train.shape[0]))
    UpdateAlfa = list(np.zeros(data_train.shape[0]))

    baris = matrix_hessian.shape[0]
    kolom = matrix_hessian.shape[1]

    while z < maxIter:
        
        #Menghitung Nilai Error
        nilaiE = np.zeros(baris)
        for i in range(baris):
            total_error = 0
            for j in range(kolom):
                error = (a_star_init[j] - a_init[j]) * matrix_hessian[i][j]
                total_error += error
            nilaiE[i] = datay_train[i] - total_error

        #Menghitung Nilai Delta a* Delta a
        maxR = max(matrix_hessian.diagonal())
        gamma = cLR/maxR
        
        for i in range(baris):
            hasilDalfaS[i] = min(max(gamma*(nilaiE[i]-epsilon), -a_star_init[i]), C - a_star_init[i])
            hasilDalfa[i] = min(max(gamma*(-nilaiE[i]-epsilon), -a_init[i]), C - a_init[i])

        #Menghitung Nilai a* dan a
        for i in range(baris):
            UpdateAlfaS[i] = a_star_init[i] + hasilDalfaS[i]
            UpdateAlfa[i] = a_init[i] + hasilDalfa[i]
        
        a_star_init = UpdateAlfaS
        a_init = UpdateAlfa
        z+=1
        
        #Do Something with Trigger
        if max(abs(np.asarray(hasilDalfaS))) < epsilon and max(abs(np.asarray(hasilDalfa))) < epsilon:
            break

    return a_star_init, a_init

#Hitung hasil regresi
def fungsiRegresi(data, matrix_hessian, a_star, a):
    # print(matrix_hessian)
    baris = data.shape[0]
    sum_fx = list()
    
    for i in range(baris):
        sum_fx.append(sum([(a_star[j] - a[j])*matrix_hessian[i][j] for j in range(baris)]))
    # print(sum_fx)
    return sum_fx

#Plotting
def plot(y_test, y_pred):
    # y_test = y_test.reset_index()
    # y_pred = y_pre
    plt.plot(np.arange(len(y_test)),y_test)
    plt.plot(np.arange(len(y_pred)),y_pred)
    plt.legend(['y_test','y_pred'])
    plt.show()

In [ ]:
mape = 1000

param_grid_poly = {
    'C':[0.1, 0.2, 0.5, 1, 3, 10, 50, 100, 150],
    'cLR':[0.0001, 0.004, 0.007, 0.005, 0.05, 0.1, 0.5, 0.7, 0.9],
    'epsilon':[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.5],
    'lamda':[0.001, 0.01, 0.1, 1, 3, 5, 10, 15, 20],
    'd':[1,2,3,4,5,6,7,8,9],
    'kernel':['poly']
}

X = data_raw.iloc[:,2:8]
Y = data_raw['Selisih']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, shuffle=False)

X_train_scaled, XTrain_scaler = normalisasi(X_train)
y_train_scaled, YTrain_scaler = normalisasi(y_train.values.reshape(-1,1))
X_test_scaled = XTrain_scaler.transform(X_test)

parameter_looping_poly = ParameterGrid(param_grid_poly)

for task in parameter_looping_poly:

    hessian = matrixHessian(X_train_scaled, X_train_scaled, task['lamda'],task['d'])
    a_bintang, a_biasa = sequentialLearning(X_train_scaled, y_train_scaled, hessian, epsilon=task['epsilon'], C = task['C'], cLR = task['cLR'])
    hessian_test = matrixHessian(X_train_scaled, X_test_scaled, task['lamda'], task['d'])
    result = fungsiRegresi(X_test_scaled, hessian_test, a_bintang, a_biasa)

    result_inverted = YTrain_scaler.inverse_transform(np.array(result).reshape(-1,1))

    mape_temp = mean_absolute_percentage_error(y_test,result_inverted)
    if mape_temp < mape:
        print("mape = %.2f" % (mape_temp),"%")
        print(task)
        mape = mape_temp
        result_model = task

print("Best Model Acquired")
print("")
print("MAPE = %.2f" % (mape),"%")
for key, i in task.items():
    print(f"{key} : {i}")